## Домашнее задание к уроку 4 ##

1.	Сохранить данные из предыдущего домашнего задания в файл .json или .csv. 
2.	Создать MongoDB, записать данные туда (любое название базы, любое название коллекции). Выполнить команду для демонстрации содержимого коллекции. Прикрепить скриншот.
3.	Создать базу данных sqlite, загрузить туда данные из парсера с предыдущего урока. Загрузить файл .db


In [68]:
import os
import json
import requests
from bs4 import BeautifulSoup as bs

In [69]:
pip install pymongo

In [70]:
from pymongo import MongoClient

In [71]:
# 1 Сохранить данные из предыдущего домашнего задания в файл .json или .csv

def add_to_result(result, quote_text, author, tags):
    quote = {
        "text": quote_text,
        "author": author,
        "tags": tags
    }
    result.append(quote)  
    return result


In [72]:
def parce_quotes(url_to_parce, result):
    url_to_parce
    r = requests.get(url_to_parce)
    
    html = bs(r.content, 'html.parser')
    all_quotes = html.find_all('div', attrs={"class": "quote"})

    for quote in all_quotes:
        quote_text = quote.find('span', attrs={"itemprop": "text"}).text
        author = quote.find(attrs={"itemprop": "author"}).text
        tags = quote.find(attrs={"itemprop": "keywords"})['content']
    
        result = add_to_result(result, quote_text, author, tags)
        
    pager = html.find('ul', attrs={"class": "pager"})
    if pager != None:
        next_page = pager.find('li', attrs={"class": "next"})
        if next_page != None:
            url_postfix = next_page.find('a')['href']
        else:
            url_postfix = None            
    else:
        url_postfix = None    
    
    return {
        'result': result,
        'url_psfx': url_postfix
    }

In [ ]:
url = 'https://quotes.toscrape.com/'
hw_result = []
url_psfx = '/'
url_prc = url[:-1] + url_psfx

url_prc

while url_psfx != None:
    url_prc = url[:-1] + url_psfx
    
    result = parce_quotes(url_prc, hw_result)
    url_psfx = result['url_psfx']
    #print(url_psfx)
    
    hw_result = result['result']
    
hw_result

In [ ]:
json.dump(hw_result, fp = open("result_hw3.txt", "w+"))

with open('result_hw3.txt') as f:
    quotes = json.load(f)
    
quotes

In [ ]:
# 2. Создать MongoDB, записать данные туда (любое название базы, любое название коллекции). 
# Выполнить команду для демонстрации содержимого коллекции. Прикрепить скриншот.

client = MongoClient('mongodb://localhost:27017/')

In [ ]:
print(client.list_database_names())

In [ ]:
db = client.quotes

In [ ]:
db.quotes.insert_many(quotes)

In [ ]:
print(client.list_database_names())

In [ ]:
print(db.list_collection_names())

In [ ]:
quotes_in_base = db.quotes.find()
for quote in quotes_in_base:
    print('{0} {1}'.format(quote['text'], 
        quote['author']))

In [ ]:
# 3. Создать базу данных sqlite, загрузить туда данные из парсера с предыдущего урока. Загрузить файл .db

import sqlite3

In [ ]:
con = sqlite3.connect('sqlite.db')
cursor = con.cursor()

In [ ]:
query = '''
    DROP TABLE IF EXISTS quotes;
    '''
cursor.execute(query)
con.commit()

In [ ]:
query = '''
    CREATE TABLE  quotes(
        quote_text CHAR,
        author CHAR,
        tags CHAR
    )
'''
cursor.execute(query)
con.commit()

In [ ]:
insert_query = '''
    INSERT INTO quotes(
        quote_text,
        author,
        tags
    )
    VALUES(
        ?,
        ?,
        ?
    )
'''

for quote in quotes:
    quote_tuple = (quote['text'], quote['author'], quote['tags'])
    cursor.execute(insert_query, quote_tuple)
    con.commit()

In [ ]:
cursor.execute('SELECT * FROM quotes')
find_quotes = cursor.fetchall()
for quote in find_quotes:
    print(quote)

In [ ]:
bck = sqlite3.connect('quotes.db')
with bck:
    con.backup(bck, pages=1)
bck.close()

In [ ]:
con.close()